Neural Network from Scratch 

In [1]:
import numpy as np 

In [2]:
class Op:
    
    def __init__(self, Inputs=[]):
        
        self.Operands = Inputs
        
        self.Operation2Perform = []
        
        for SingleOp in Inputs:
            
            SingleOp.Operation2Perform.append(self)
            
        _default_graph.Ops.append(self)
        
            
    def ComputeOp(self):
        
        pass

In [3]:
class Add(Op):
    
    def __init__(self,FirstOperand, SecondOperand):
        
        super().__init__([FirstOperand, SecondOperand])
        
    def ComputeOp(self,FirstOperand, SecondOperand):
        
        self.InputOperands = [FirstOperand, SecondOperand]
        
        return (FirstOperand + SecondOperand)

In [4]:
class Mul(Op):
    
    def __init__(self,FirstOperand, SecondOperand):
        
        super().__init__([FirstOperand, SecondOperand])
        
    def ComputeOp(self,FirstOperand, SecondOperand):
        
        self.InputOperands = [FirstOperand, SecondOperand]
        
        return (FirstOperand * SecondOperand)

In [5]:
class MatMul(Op):
    
    def __init__(self,FirstOperand, SecondOperand):
        
        super().__init__([FirstOperand, SecondOperand])
        
    def ComputeOp(self,FirstOperand, SecondOperand):
        
        self.InputOperands = [FirstOperand, SecondOperand]
        
        return (np.dot(FirstOperand, SecondOperand))

In [6]:
class Placeholder:
    
    def __init__(self):
        
        self.Operation2Perform = []
        
        _default_graph.Plchldrs.append(self)

In [7]:
class Variable:
    
    def __init__(self,Thetas=None):
        
        self.InitialThetas = Thetas
        
        self.Operation2Perform = []
        
        _default_graph.Vars.append(self)

In [8]:
class Graph:
    
    def __init__(self):
        
        self.Ops = []
        self.Plchldrs = []
        self.Vars = []
        
    def set_as_default(self):
        
        global _default_graph
        
        _default_graph = self

In [9]:
def EachOpComponentsPostOrderTraversal(CurrentOp):
    
    OpComponentsInPostOrder = []
    
    def RecursivePostOrdering(CurrentOp):
    
        if isinstance(CurrentOp,Op):

            for EachComponent in CurrentOp.Operands:

                RecursivePostOrdering(EachComponent)

        OpComponentsInPostOrder.append(CurrentOp)
            
            
            
    RecursivePostOrdering(CurrentOp)
    
    return OpComponentsInPostOrder

In [10]:
class Session:
    
    def run(self,CurrentOp,feed_dict={}):
        
        OpComponentsInPostOrder = EachOpComponentsPostOrderTraversal(CurrentOp)
        
        for EachComponent in OpComponentsInPostOrder:
            
            if type(EachComponent) == Variable:
                
                EachComponent.Output = EachComponent.InitialThetas
                
            elif type(EachComponent) == Placeholder:
                
                EachComponent.Output = feed_dict[EachComponent]
                
            else:
                
                EachComponent.Inputs = [EachOperand.Output for EachOperand in EachComponent.Operands]
                
                EachComponent.Output = EachComponent.ComputeOp(*EachComponent.Inputs)
                
                EachComponent.Output = np.array(EachComponent.Output)
                
        return CurrentOp.Output

Lets Compute the expression, y = mx + c through our code, where m  = 75 and c = 36 and we can change 'x'

Ideally, we would have taken 'x' as an object of 'Variable' class because obviously in normal mathematics also, we take 'x' as 'Variable' and 'm' and 'c' as constants which in our programming environments stand for 2 objects of 'Placeholder' class. 

But, we are going to do the role reversal. Why ???? Because if you guys remember during linear regression, we took 'm' as 'Theta1' and 'c' as 'Theta0'. Now, in TensorFlow, we take those components of operation as belonging to 'Variable' class which are continously changing. 

So, we can clearly see that in our neural network, Thetas are changing and Data which we are feeding is not changing. So, thats what we are going to do: Role Reversal. 

We are going to consider 'm' and 'c' as belonging to 'Variable' class and 'x' like a data which remains constant during backward and forward passes, belonging to 'Placeholder' class. 

Now lets first compute 'FirstOperation = 'm' (Which is a 'Variable' object) * (Which is Mul(Op) object) 'x' (Which is a 'Placeholder' object)

Now what will happen once we execute this. Well, in this scenario, 'FirstOperation' is going to be an object of class 'Mul(Op)'. How ???? Lets leave it aside for sometime that how ?? We will come back later to this. 

So, in order to perform our first operation, we first need to create an object 'm' of class 'Variable' and object 'x' of class 'Placeholder' and then use both of them to create an object 'FirstOperation' of class 'Add(Op)' by calling the constructor '__init__(Firstoperand,SecondOperand)' of 'Add(Op)' class where 'FirstOperand' is going to be 'm' and 'SecondOperand' is going to be 'x'. 

In [11]:
g = Graph ()

In [12]:
g.set_as_default()

In [13]:
m = Variable(75)

In [14]:
x = Placeholder()

Why we haven't given any value to the placeholder ???? Because 'Placeholder'(s) are for feeding data. And in TensorFlow, data is always feeded at the end. In our current context, data will be only feeded or the value of 'x' will be only feeded once we have created our complete neural network whose output is '75x + 36' and then we feed the value of 'x'. 

In [15]:
FirstOperation  = Mul(m,x)

In [16]:
c = Variable(36)

In [17]:
SecondOperation = Add(FirstOperation,c)

In [18]:
S = Session ()

In [19]:
FinalResult = S.run(CurrentOp=SecondOperation,feed_dict={x:12})

In [20]:
print(FinalResult)

936
